<h1>Part A</h1>

<h2>Task 1 MongoDB data model</h2>

<h3>Schema Design Rationale</h3>
In my schema, I have 2 models. Climate Schema stores the weather data for a particular day and hotspot schema stores data about every occurance of a fire. Date is used to join associated fires and climate for a particular day. To achieve this, I made hotspot schema store 2 datetime fields, date and datetime. While datetime stores both the date and time of when the fire was reported, date will only store the date value. This makes join queries easier between climate and hotspot data. Another alternative could be to store the object reference of a climate record. However I preferred this schema because it allows for better flexibility in adding fire data independently to the database. This allows the schema to be eaasily altered for future use. Also hotspot data does not need to wait for an existing climate data before adding a new record to the collection. Compared to nesting hotspot data within a climate schema, queries are more efficient in most cases when data needs to be queried over multiple days because there is only 1 level to search over and a smaller search space due to dividing data across 2 collections. It also keeps queries less complex

In [36]:
ClimateSchema = [
    {
        "date": "",
        "station": "",
        "relative_humdity":"",
        "windspeed_knots": "",
        "air_temperature_celsius":"",
        "max_wind_speed": "",
        "measure": "",
        "flag": "",
        "GHI_w/m2": "",
    }
]

hotspotSchema = [
    {
        "date": "",
        "datetime": "",
        "lattitude": "",
        "longitude": "",
        "confidence": "",
        "surface_temperature_celsius": "",
        
    }
 ]



<h2>Task 2: Queries MongoDB</h2>

Data is imported from CSV files 

In [33]:
import pandas as pd
import re
climateCsv = pd.read_csv("./climate_historic.csv")
hotspotCsv = pd.read_csv("./hotspot_historic.csv")


Connect to mongodb client 
and drop existing collection

In [34]:
from datetime import datetime

import pymongo
from pymongo import MongoClient
client = MongoClient('192.168.0.108', 27017) #connect to local ip

db = client.fit3182_db 
climate = db["climate"]
hotspots = db['hotspots']
climate.drop() #drop database
hotspots.drop()

<h3>Insert Climate data</h3>

In [40]:

for index, row in climateCsv.iterrows():
    dateStr = row["date"]
    date = datetime.strptime(dateStr, '%d/%m/%Y')  # Convert date string to datetime object
    station = row['station']
    precipitation = row['precipitation']
    result = re.match(r"(\d+\.\d+)([A-Za-z]+)", precipitation)
    numeric_part = float(result.group(1))
    alphabetic_part = result.group(2)
    air_temperature_celcius = row['air_temperature_celcius']
    relative_humidity = row['relative_humidity']
    windspeed_knots = row['windspeed_knots']
    GHI = row["GHI_w/m2"]
    max_wind_speed = row['max_wind_speed']
    
    newClimate = {
        "date": date,
        "station": station,
        "relative_humdity": relative_humidity,
        "air_temperature_celsius": air_temperature_celcius,
        "windspeed_knots": windspeed_knots,
        "max_wind_speed": max_wind_speed,
        "measure": numeric_part,
        "flag": alphabetic_part,
        "GHI_w/m2": GHI,
    }
    result = climate.insert_one(newClimate)


<h3>Insert hotspot data</h3>

In [41]:
for index, row in hotspotCsv.iterrows():
    dateStr = row["date"]
    date = datetime.strptime(dateStr, '%d/%m/%Y')  # Convert date string to datetime object
    dateTimeStr = row["datetime"]
    timeStr = row["datetime"]
    datetime_obj = datetime.fromisoformat(timeStr)
    latitude = row['latitude']
    longitude = row['longitude']
    confidence = row['confidence']
    surface_temperature_celsius = row['surface_temperature_celcius']
    
    
    newHotspot = {
        "date": date,
        "datetime": datetime_obj,
        "latitude": latitude,
        "longitude": longitude,
        "confidence": confidence,
        "surface_temperature_celsius": surface_temperature_celsius,
    }
    
    # Insert the new record into MongoDB
    result = hotspots.insert_one(newHotspot)
    

<h2>2 Queries</h2>
a) Find climate data on 12th December 2022

In [61]:
#use datetime to quer
query_date = datetime(2022, 1, 20)

result = climate.find({"date": {"$eq": query_date}})

for data in result:
    data["date"] = data["date"].strftime("%Y-%m-%d")
    print(data)

{'_id': ObjectId('648c331fd2b9e8cbf0a645a5'), 'date': '2022-01-20', 'station': 948700, 'relative_humdity': 50.9, 'air_temperature_celsius': 16, 'windspeed_knots': 12.9, 'max_wind_speed': 21.0, 'measure': 0.03, 'flag': 'G', 'GHI_w/m2': 136}


b) Find the latitude, longitude, surface temperature (°C), and confidence when the surface temperature (°C) was between 65 °C and 100 °C.

In [43]:

# Task 2: Find the latitude, longitude, surface temperature (°C), and confidence
# when the surface temperature (°C) was between 65 °C and 100 °C

result = hotspots.find({
    "surface_temperature_celsius": {"$gte": 65, "$lte": 100}
}, {"latitude": 1, "longitude": 1, "surface_temperature_celsius": 1, "confidence": 1})


for doc in result:
    print(doc)

{'_id': ObjectId('648c3321d2b9e8cbf0a64700'), 'latitude': -37.966, 'longitude': 145.051, 'confidence': 78, 'surface_temperature_celsius': 68}
{'_id': ObjectId('648c3321d2b9e8cbf0a64703'), 'latitude': -35.543, 'longitude': 143.316, 'confidence': 86, 'surface_temperature_celsius': 67}
{'_id': ObjectId('648c3321d2b9e8cbf0a6470a'), 'latitude': -37.875, 'longitude': 142.51, 'confidence': 93, 'surface_temperature_celsius': 73}
{'_id': ObjectId('648c3321d2b9e8cbf0a6470c'), 'latitude': -37.613, 'longitude': 149.305, 'confidence': 95, 'surface_temperature_celsius': 75}
{'_id': ObjectId('648c3321d2b9e8cbf0a6470e'), 'latitude': -37.624, 'longitude': 149.314, 'confidence': 90, 'surface_temperature_celsius': 66}
{'_id': ObjectId('648c3321d2b9e8cbf0a64711'), 'latitude': -38.057, 'longitude': 144.211, 'confidence': 93, 'surface_temperature_celsius': 73}
{'_id': ObjectId('648c3321d2b9e8cbf0a6471a'), 'latitude': -37.95, 'longitude': 142.366, 'confidence': 92, 'surface_temperature_celsius': 70}
{'_id': 

c) Find date, surface temperature (°C), air temperature (°C), relative humidity and max wind speed on 15th and 16th of December 2022. 


In [59]:

start_date = datetime(2022, 12, 15, 0, 0, 0)
end_date = datetime(2022, 12, 16, 23, 59, 59)
result = climate.find({
    "date": {"$in": [datetime(2022, 12, 15), datetime(2022, 12, 16)]}
}, {"date": 1, "surface_temperature_celsius": 1, "air_temperature_celsius": 1, "relative_humidity": 1, "max_wind_speed": 1})


for data in result:
    data["date"] = data["date"].strftime("%Y-%m-%d")
    print(doc)


{'totalFires': 2, 'date': '2022-07-31'}
{'totalFires': 2, 'date': '2022-07-31'}


d) Find datetime, air temperature (°C), surface temperature (°C),
and confidence when the confidence is between 80 and 100

In [58]:

result = hotspots.find({
    "confidence": {"$gte": 80, "$lte": 100}
}, {
    "datetime": 1,
    "surface_temperature_celsius": 1,
    "air_temperature_celsius": 1,
    "confidence": 1
}
)

for data in result:
    data["datetime"] = data["datetime"].strftime("%Y-%m-%d %H:%M:%S")
    print(data)


{'_id': ObjectId('648c3321d2b9e8cbf0a64701'), 'datetime': '2022-12-27 00:02:15', 'confidence': 82, 'surface_temperature_celsius': 63}
{'_id': ObjectId('648c3321d2b9e8cbf0a64703'), 'datetime': '2022-12-27 00:02:14', 'confidence': 86, 'surface_temperature_celsius': 67}
{'_id': ObjectId('648c3321d2b9e8cbf0a64704'), 'datetime': '2022-12-25 04:29:08', 'confidence': 80, 'surface_temperature_celsius': 54}
{'_id': ObjectId('648c3321d2b9e8cbf0a64707'), 'datetime': '2022-12-16 15:38:39', 'confidence': 94, 'surface_temperature_celsius': 43}
{'_id': ObjectId('648c3321d2b9e8cbf0a6470a'), 'datetime': '2022-12-16 04:35:13', 'confidence': 93, 'surface_temperature_celsius': 73}
{'_id': ObjectId('648c3321d2b9e8cbf0a6470b'), 'datetime': '2022-12-16 04:34:58', 'confidence': 84, 'surface_temperature_celsius': 55}
{'_id': ObjectId('648c3321d2b9e8cbf0a6470c'), 'datetime': '2022-12-16 04:34:58', 'confidence': 95, 'surface_temperature_celsius': 75}
{'_id': ObjectId('648c3321d2b9e8cbf0a6470e'), 'datetime': '202

e)Find the top 10 records with the highest surface temperature (°C).


In [63]:
result = hotspots.find().sort("surface_temperature_celsius", -1).limit(10)

for data in result:
    data["datetime"] = data["datetime"].strftime("%Y-%m-%d %H:%M:%S")
    data["date"] = data["date"].strftime("%Y-%m-%d")

    print(data)


{'_id': ObjectId('648c3322d2b9e8cbf0a64ba3'), 'date': '2022-04-18', 'datetime': '2022-04-18 04:52:00', 'latitude': -38.1665, 'longitude': 143.062, 'confidence': 100, 'surface_temperature_celsius': 124}
{'_id': ObjectId('648c3323d2b9e8cbf0a6504d'), 'date': '2022-04-04', 'datetime': '2022-04-04 04:32:50', 'latitude': -36.343, 'longitude': 142.1986, 'confidence': 100, 'surface_temperature_celsius': 123}
{'_id': ObjectId('648c3322d2b9e8cbf0a64b15'), 'date': '2022-05-01', 'datetime': '2022-05-01 04:14:20', 'latitude': -36.9318, 'longitude': 143.0907, 'confidence': 100, 'surface_temperature_celsius': 122}
{'_id': ObjectId('648c3324d2b9e8cbf0a6513c'), 'date': '2022-03-18', 'datetime': '2022-03-18 03:50:50', 'latitude': -37.017, 'longitude': 148.1297, 'confidence': 100, 'surface_temperature_celsius': 121}
{'_id': ObjectId('648c3322d2b9e8cbf0a64927'), 'date': '2022-05-13', 'datetime': '2022-05-13 04:40:20', 'latitude': -34.9938, 'longitude': 141.876, 'confidence': 100, 'surface_temperature_cels

f) Find the number of fires each day. You are required to only display the total number of fires and the date in the output.

In [64]:
from collections import defaultdict

# Perform the aggregation
pipeline = [
    {
        '$group': {
            '_id': {
                '$dateToString': {
                    'format': '%Y-%m-%d',
                    'date': '$date'
                }
            },
            'totalFires': { '$sum': 1 }
        }
    },
    {
        '$project': {
            '_id': 0,
            'date': '$_id',
            'totalFires': 1
        }
    }
]

result = hotspots.aggregate(pipeline)
# Display the output
for doc in result:
    print('Date: {}, Total Fires: {}'.format(doc['date'], doc['totalFires']))

Date: 2022-05-15, Total Fires: 102
Date: 2022-04-25, Total Fires: 3
Date: 2022-04-12, Total Fires: 69
Date: 2022-07-01, Total Fires: 4
Date: 2022-12-08, Total Fires: 5
Date: 2023-10-21, Total Fires: 1
Date: 2023-12-12, Total Fires: 3
Date: 2022-11-29, Total Fires: 8
Date: 2022-05-22, Total Fires: 33
Date: 2023-12-21, Total Fires: 2
Date: 2022-05-14, Total Fires: 1
Date: 2022-08-05, Total Fires: 1
Date: 2022-11-14, Total Fires: 3
Date: 2022-03-29, Total Fires: 1
Date: 2022-10-21, Total Fires: 4
Date: 2022-09-21, Total Fires: 2
Date: 2023-10-25, Total Fires: 1
Date: 2022-11-12, Total Fires: 5
Date: 2022-05-16, Total Fires: 3
Date: 2022-05-13, Total Fires: 54
Date: 2022-08-01, Total Fires: 2
Date: 2022-04-20, Total Fires: 31
Date: 2023-12-08, Total Fires: 1
Date: 2022-10-10, Total Fires: 3
Date: 2022-09-22, Total Fires: 1
Date: 2022-10-09, Total Fires: 1
Date: 2022-12-25, Total Fires: 1
Date: 2022-10-04, Total Fires: 5
Date: 2022-04-17, Total Fires: 38
Date: 2022-10-01, Total Fires: 8
Dat

g) Find the records of fires where the confidence is below 70.

In [66]:

# Find the records of fires where the confidence is below 70
result = hotspots.find({"confidence": {"$lt": 70}})

for data in result:
    data["datetime"] = data["datetime"].strftime("%Y-%m-%d %H:%M:%S")
    data["date"] = data["date"].strftime("%Y-%m-%d")
    print(data)

{'_id': ObjectId('648c2f80fed0caa0de397cef'), 'type': 'other', 'date': '2023-10-18', 'datetime': '2023-10-18 09:36:00', 'surface_temperature_celsius': 24.0, 'confidence': 37.0, 'latitude': -18.7366, 'longitude': 71.59385}
{'_id': ObjectId('648c2f80fed0caa0de397cf0'), 'type': 'other', 'date': '2023-10-18', 'datetime': '2023-10-18 14:24:00', 'surface_temperature_celsius': 22.5, 'confidence': 31.5, 'latitude': -18.93795, 'longitude': 71.65705}
{'_id': ObjectId('648c2f80fed0caa0de397cf1'), 'type': 'other', 'date': '2023-10-18', 'datetime': '2023-10-18 14:24:00', 'surface_temperature_celsius': 25.0, 'confidence': 38.5, 'latitude': -18.8418, 'longitude': 71.30345}
{'_id': ObjectId('648c2f8afed0caa0de397cf5'), 'type': 'natural', 'date': '2023-10-19', 'datetime': '2023-10-19 14:24:00', 'surface_temperature_celsius': 30.0, 'confidence': 43.0, 'latitude': -18.451, 'longitude': 70.99825}
{'_id': ObjectId('648c2f94fed0caa0de397cf9'), 'type': 'other', 'date': '2023-10-20', 'datetime': '2023-10-20 1

h) Find the average surface temperature (°C) for each day. You are required to only display average surface temperature (°C) and the date in the output. 


In [67]:
from collections import defaultdict

pipeline = [
    {
        '$group': {
            '_id': { '$dateToString': { 'format': '%Y-%m-%d', 'date': '$date' } },
            'averageSurfaceTemp': { '$avg': '$surface_temperature_celsius' }
        }
    },
    {
        '$project': {
            'date': '$_id',
            'averageSurfaceTemp': 1,
            '_id': 0
        }
    }
]

result = hotspots.aggregate(pipeline)

for doc in result:
    print(doc)


{'averageSurfaceTemp': 53.950980392156865, 'date': '2022-05-15'}
{'averageSurfaceTemp': 48.666666666666664, 'date': '2022-04-25'}
{'averageSurfaceTemp': 52.69565217391305, 'date': '2022-04-12'}
{'averageSurfaceTemp': 50.6, 'date': '2022-12-08'}
{'averageSurfaceTemp': 30.5, 'date': '2022-07-01'}
{'averageSurfaceTemp': 32.0, 'date': '2023-10-21'}
{'averageSurfaceTemp': 60.625, 'date': '2022-11-29'}
{'averageSurfaceTemp': 22.833333333333332, 'date': '2023-12-12'}
{'averageSurfaceTemp': 54.484848484848484, 'date': '2022-05-22'}
{'averageSurfaceTemp': 30.25, 'date': '2023-12-21'}
{'averageSurfaceTemp': 49.0, 'date': '2022-05-14'}
{'averageSurfaceTemp': 40.0, 'date': '2022-08-05'}
{'averageSurfaceTemp': 52.0, 'date': '2022-11-14'}
{'averageSurfaceTemp': 51.0, 'date': '2022-03-29'}
{'averageSurfaceTemp': 51.25, 'date': '2022-10-21'}
{'averageSurfaceTemp': 40.5, 'date': '2022-09-21'}
{'averageSurfaceTemp': 22.0, 'date': '2023-10-25'}
{'averageSurfaceTemp': 53.0, 'date': '2022-11-12'}
{'average

i) Find the top 10 records with the lowest GHI.


In [69]:

# Find the top 10 records with the lowest GHI
result = climate.find().sort("GHI_w/m2", 1).limit(10)

for data in result:
    data["date"] = data["date"].strftime("%Y-%m-%d")
    print(data)

{'_id': ObjectId('648c331fd2b9e8cbf0a64667'), 'date': '2022-08-02', 'station': 948701, 'relative_humdity': 38.6, 'air_temperature_celsius': 5, 'windspeed_knots': 1.8, 'max_wind_speed': 5.1, 'measure': 0.0, 'flag': 'I', 'GHI_w/m2': 47}
{'_id': ObjectId('648c331fd2b9e8cbf0a64646'), 'date': '2022-06-30', 'station': 948701, 'relative_humdity': 34.9, 'air_temperature_celsius': 5, 'windspeed_knots': 4.3, 'max_wind_speed': 11.1, 'measure': 0.0, 'flag': 'A', 'GHI_w/m2': 48}
{'_id': ObjectId('648c331fd2b9e8cbf0a64647'), 'date': '2022-07-01', 'station': 948701, 'relative_humdity': 33.1, 'air_temperature_celsius': 5, 'windspeed_knots': 5.8, 'max_wind_speed': 14.0, 'measure': 0.0, 'flag': 'I', 'GHI_w/m2': 49}
{'_id': ObjectId('648c331fd2b9e8cbf0a64651'), 'date': '2022-07-11', 'station': 948701, 'relative_humdity': 39.0, 'air_temperature_celsius': 6, 'windspeed_knots': 1.8, 'max_wind_speed': 5.1, 'measure': 0.01, 'flag': 'G', 'GHI_w/m2': 56}
{'_id': ObjectId('648c331fd2b9e8cbf0a64655'), 'date': '20

j) Find the records with a 24-hour precipitation recorded between 0.20 to 0.35.


In [70]:
result = climate.find({
    "measure": {"$gte": 0.20, "$lte": 0.35}, "flag": {"$eq": 'G'}
})


for doc in result:
    print(doc)

{'_id': ObjectId('648c2f9efed0caa0de397cfd'), 'date': datetime.datetime(2023, 10, 24, 0, 0), 'station': 9920, 'relative_humidity': 40.8, 'air_temperature_celsius': 11, 'windspeed_knots': 12.2, 'max_wind_speed': 20.0, 'measure': 0.24, 'flag': 'G', 'GHI_w/m2': 102}
{'_id': ObjectId('648c3084fed0caa0de397d47'), 'date': datetime.datetime(2023, 11, 16, 0, 0), 'station': 443, 'relative_humidity': 46.2, 'air_temperature_celsius': 11, 'windspeed_knots': 10.5, 'max_wind_speed': 15.0, 'measure': 0.28, 'flag': 'G', 'GHI_w/m2': 97}
{'_id': ObjectId('648c3098fed0caa0de397d4f'), 'date': datetime.datetime(2023, 11, 18, 0, 0), 'station': 123, 'relative_humidity': 45.1, 'air_temperature_celsius': 11, 'windspeed_knots': 11.5, 'max_wind_speed': 18.1, 'measure': 0.2, 'flag': 'G', 'GHI_w/m2': 98}
{'_id': ObjectId('648c3192fed0caa0de397d9d'), 'date': datetime.datetime(2023, 12, 13, 0, 0), 'station': 6558, 'relative_humidity': 38.6, 'air_temperature_celsius': 8, 'windspeed_knots': 12.8, 'max_wind_speed': 18.

In [72]:
climate.drop() #drop database
hotspots.drop()